Potentially Hazardous Asteroids (PHAs) are currently defined based on parameters that measure the asteroid's potential to make threatening close approaches to the Earth. Specifically, all asteroids with an Earth Minimum Orbit Intersection Distance (MOID) of 0.05 au or less and an absolute magnitude (H) of 22.0 or less are considered PHAs. In other words, asteroids that can't get any closer to the Earth (i.e., MOID) than 0.05 au (roughly 7,480,000 km or 4,650,000 mi) or are smaller than about 140 m (~500 ft) in diameter (i.e., H = 22.0 with assumed albedo of 14%) are not considered PHAs.

In [1]:
import requests
from urllib.request import urlretrieve
from pprint import PrettyPrinter
pp = PrettyPrinter()
import pandas as pd
import json
from datetime import timedelta, date
import configparser
import pickle

In [2]:
# Setting Column widthsize to view all columns in pandas data frame
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
config = configparser.ConfigParser()
config.read('dl.cfg')

['dl.cfg']

In [5]:
apiKey = config['NASA']['apikey']


# **NEO FEED**

### **Sample Output**
`{'absolute_magnitude_h': 19.88,
 'close_approach_data': [{'close_approach_date': '2021-11-10',
                          'close_approach_date_full': '2021-Nov-10 20:29',
                          'epoch_date_close_approach': 1636576140000,
                          'miss_distance': {'astronomical': '0.2306734775',
                                            'kilometers': '34508260.899492925',
                                            'lunar': '89.7319827475',
                                            'miles': '21442439.023835365'},
                          'orbiting_body': 'Earth',
                          'relative_velocity': {'kilometers_per_hour': '46525.6340756068',
                                                'kilometers_per_second': '12.9237872432',
                                                'miles_per_hour': '28909.219684368'}}],
 'estimated_diameter': {'feet': {'estimated_diameter_max': 2060.7486784504,
                                 'estimated_diameter_min': 921.5948259116},
                        'kilometers': {'estimated_diameter_max': 0.6281161771,
                                       'estimated_diameter_min': 0.2809020939},
                        'meters': {'estimated_diameter_max': 628.116177092,
                                   'estimated_diameter_min': 280.902093949},
                        'miles': {'estimated_diameter_max': 0.3902931771,
                                  'estimated_diameter_min': 0.174544415}},
 'id': '2068347',
 'is_potentially_hazardous_asteroid': True,
 'is_sentry_object': False,
 'links': {'self': 'http://www.neowsapp.com/rest/v1/neo/2068347?api_key=tydbiYxRDkZwIRlMQaAilILralTGaivulTvpTnht'},
 'name': '68347 (2001 KB67)',
 'nasa_jpl_url': 'http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2068347',
 'neo_reference_id': '2068347'}`

In [10]:
with open("rawdata.pickle", "wb") as f:   #Saving the data locally as a text file to be reused
   pickle.dump(data_list, f)

In [11]:
with open("rawdata.pickle", "rb") as f:   # Loading the data collected 
   raw = pickle.load(f)

In [23]:
df = pd.DataFrame(raw)

In [54]:
df.isnull().sum()

links                                0
id                                   0
neo_reference_id                     0
name                                 0
nasa_jpl_url                         0
absolute_magnitude_h                 0
estimated_diameter                   0
is_potentially_hazardous_asteroid    0
close_approach_data                  0
is_sentry_object                     0
estimated_diameter_min_km            0
estimated_diameter_max_km            0
close_approach_date                  0
miss_distance_km                     0
relative_velocity_kmps               0
orbiting_body                        0
dtype: int64

In [38]:
#Droping Null Values
df.dropna(inplace=True)

In [39]:
# Getting Data from nested dictionaries of certain columns

df['estimated_diameter_min_km'] = df['estimated_diameter'].apply(lambda x: x['kilometers']['estimated_diameter_min'])

df['estimated_diameter_max_km'] = df['estimated_diameter'].apply(lambda x: x['kilometers']['estimated_diameter_max'])

df['close_approach_date'] = df['close_approach_data'].apply(lambda x: x[0]['close_approach_date'])

df['miss_distance_km'] = df['close_approach_data'].apply(lambda x: x[0]['miss_distance']['kilometers'])

df['relative_velocity_kmps'] = df['close_approach_data'].apply(lambda x: x[0]['relative_velocity']['kilometers_per_second'])

df['orbiting_body'] = df['close_approach_data'].apply(lambda x: x[0]['orbiting_body'])

In [45]:
df.dtypes

links                                 object
id                                    object
neo_reference_id                      object
name                                  object
nasa_jpl_url                          object
absolute_magnitude_h                 float64
estimated_diameter                    object
is_potentially_hazardous_asteroid       bool
close_approach_data                   object
is_sentry_object                        bool
estimated_diameter_min_km            float64
estimated_diameter_max_km            float64
close_approach_date                   object
miss_distance_km                      object
relative_velocity_kmps                object
orbiting_body                         object
dtype: object

In [46]:
df2.head(1)

,links,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,estimated_diameter,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,estimated_diameter_min_km,estimated_diameter_max_km,close_approach_date,miss_distance_km,relative_velocity_kmps,orbiting_body
0,{'self': 'http://www.neowsapp.com/rest/v1/neo/...,2053789,2053789,53789 (2000 ED104),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2053789,17.33,{'kilometers': {'estimated_diameter_min': 0.90...,True,"[{'close_approach_date': '2002-10-09', 'close_...",False,0.908981,2.032544,2002-10-09,24798104.578835531,21.5748905048,Earth


In [47]:
#df_na4[['year','month','day']] = df_na4[['year','month','day']].apply(pd.to_numeric)
df[['id','neo_reference_id','miss_distance_km','relative_velocity_kmps',]] =df[['id','neo_reference_id','miss_distance_km','relative_velocity_kmps',]].apply(pd.to_numeric)

In [51]:
df[['close_approach_date']] = df[['close_approach_date']].apply(pd.to_datetime)

In [58]:
clean = df.drop(columns=(['links','neo_reference_id','nasa_jpl_url','estimated_diameter','close_approach_data']))
clean.head(5)

,id,name,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_min_km,estimated_diameter_max_km,close_approach_date,miss_distance_km,relative_velocity_kmps,orbiting_body
0,2053789,53789 (2000 ED104),17.33,True,False,0.908981,2.032544,2002-10-09,2.479810e+07,21.574891,Earth
1,2338292,338292 (2002 UA31),19.02,False,False,0.417402,0.933340,2002-10-09,3.374388e+07,17.325734,Earth
2,3251590,(2004 RC80),24.40,False,False,0.035039,0.078350,2002-10-09,6.481196e+07,17.037627,Earth
3,3255465,(2004 SC56),22.70,False,False,0.076658,0.171412,2002-10-09,6.111675e+06,11.170991,Earth
4,3559741,(2011 EE41),24.30,False,False,0.036691,0.082043,2002-10-09,3.462083e+07,9.130293,Earth


In [59]:
clean['miss_distance_km']=clean['miss_distance_km'].apply(lambda x: round(x,2))

In [63]:
clean.dtypes

id                                            int64
name                                         object
absolute_magnitude_h                        float64
is_potentially_hazardous_asteroid              bool
is_sentry_object                               bool
estimated_diameter_min_km                   float64
estimated_diameter_max_km                   float64
close_approach_date                  datetime64[ns]
miss_distance_km                            float64
relative_velocity_kmps                      float64
orbiting_body                                object
dtype: object

# **Code Above**
---
